# 大幅な変更があったから一旦書き直す（あとで）

# 光学的内径計測のシミュレーションを行う

- 前回シミュレーションの反省点
    - スリット通過後の受光面の位置がバラバラであった(スリット通過後のレンズによる集光を無視していた)
    - 受光面に関して分解能的概念を設けることを忘れていた -> 結果として、大きさに関わらずすべて相似形の結果が得られてしまうガバガバ設計になった
    
## 以上の反省点を活かし,新しいシミュレーション手法を考える
[概要]</br>slit,lens,P.Tを1セットとして円管周りを回転させ,目的の光をある閾値を超えた場合のみキャッチ</br>

<img src="./simulation.png" width="500"/>

## 必要となる関数
- 円管内部の屈折の状態を表す関数 calc_refraction
    - 入力：光が入射する座標, 空気と部材の屈折率
    - 出力：円管の外に出るときの座標,(Y軸となす角度(radian)) ← いらないかも
        - 光線が今どの位置(終点)にいるかの情報は返したほうが良いかも


- 受光できるかできないかを表す関数
    - 入力：外周の座標,角度
    - 出力：強度,角度,座標

 
## 検出フロー（変わったから書き直す）
1.円管内部の屈折の状態を表す関数を使って円管の端から端の範囲の状態を記録 

2.スリットの条件,エネルギーの閾値(半導体ギャップ)を設定し,-π/2<θ<π/2の範囲で回転させ条件を超えられる場合のみ記録
→正確には円管とスリットまでの距離も定義が必要


$Eg < hν = hc/λ$　を満たすかどうか

## その他必要になったら付け足す関数
- 入射角,屈折角を計算する関数 calc_angle
- 解の公式 solv_quadratic_equation
- 光線透過後の傾き計算 calc_tan_after_transparency

In [1]:
import numpy as np

#解の公式
def solv_quadratic_equation(a_q, b_q, c_q):
    # 2次方程式を解く  
    D = (b_q**2 - 4*a_q*c_q) ** (1/2)
    if D < 0:
        #接していない
        print("--要編集--")
        return 1,1
    elif D == 0:
        #接する
        print("接しています")
        return 0,0
    else:
        #2点で接する
        x_1 = (-b_q + D) / (2 * a_q)
        x_2 = (-b_q - D) / (2 * a_q)

        return x_1, x_2

# 円管内部の屈折の状態を表す関数
def calc_refraction(n1, n2, x1, y1, x2, y2, a_i, b_i, a_r, b_r):
    #屈折の様子を定義
    #1.入射した座標から屈折後の角度を求める -> スネルの法則
    sita, r_type = calc_angle(n1, n2, x1, y1, x2, y2)
    if n1 < n2: #上側のみを考えている状態 →　下側は状態が変わってくるから更に条件分岐があったほうがよい y2の位置によって変わりそう
        if y2 > 0:#y2の位置によって変わる光の屈折の状態を定義、具体的には傾きのとり方が変わる
            #n1→n2 基本的にn1<n2として考える
            #屈折角+水平軸と円の接線の法線が成す角
            #sita = calc_angle(n1, n2, x1, y1, x2, y2)
            #tan(-sita)で傾きと(x2,y2)で座標から直線の式がわかる
            #円の式と直線の式から解の公式
            #解の公式に代入する用の変数を定義
            #xが0より大きいか小さいかで傾きに-がつくか変わるtan(sita)が大きさのため
            c = tan(-sita) if x2 < 0 else c = tan(sita)
            d = y2 -c*x2
            #楕円の式 x^2/a^2 + y^2/b^2 = 1 直線の式 y = cx + d
            A = (a_i**2)*(c**2)
            B = 2*(a_i**2)*c*d
            C = (a_i**2)*(d**2) - (a_i**2)*(b_i**2)
            #解の公式で次の座標を求める
            x_q1, x_q2 = solv_quadratic_equation(A, B, C)
            #D<0の場合はもう一度 → 外周の円との交点
            if x_q1 == 1 and x_q2 == 1:
                A = (a_r**2)*(c**2)
                B = 2*(a_r**2)*c*d
                C = (a_r**2)*(d**2) - (a_r**2)*(b_r**2)
                #解の公式で次の座標を求める
                x_q1, x_q2 = solv_quadratic_equation(A, B, C)
                x_ans = x_q2
                if x_ans < 0:
                    y_ans = -np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                else:
                    y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans                
            #上側の場合はx_q2の座標を取得 -> xが負か正かで傾きが変わる #傾きで条件分岐させたほうが良いかも
            if c < 0:
                x_ans = x_q2
                y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans
            else: #c>0
                x_ans = x_q1
                y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans                
            
        else: #y2<0
            
            
    else:#n2 < n1
        if y2 > 0:                
            #n2→n1
            c = tan(-sita) if x2 < 0 else c = tan(sita)
            d = y2 -c*x2
            #全反射した場合を分ける必要がある、外周
            if r_type == 1:#全反射したとき用 たぶん最終的に外→内の光線もある
                A = (a_r**2)*(c**2)
                B = 2*(a_r**2)*c*d
                C = (a_r**2)*(d**2) - (a_r**2)*(b_r**2)
            else:
                A = (a_i**2)*(c**2)
                B = 2*(a_i**2)*c*d
                C = (a_i**2)*(d**2) - (a_i**2)*(b_i**2)
            #解の公式で次の座標を求める
            x_q1, x_q2 = solv_quadratic_equation(A, B, C)
            if c < 0:
                x_ans = x_q2
                y_ans = -np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans
            else: #c>0
                x_ans = x_q1
                y_ans = -np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans
        else: #y2<0
        

# 入射角,屈折角を計算する関数 calc_angle
def calc_angle(n1, n2, x1, y1, x2, y2, a, b):
    #楕円の接線の法線の傾き
    a = y2/(b**2)
    b = x2/(a**2)
    #光線の傾き
    c = y2 - y1
    d = x2 - x1
    sin_i = np.tan(abs(a*d - b*c)/abs(a*c - b*d))
    #光線の傾きと法線の傾きの位置関係
    if a/b < c/d:
        #光線が左側 右に法線
        position_angle = 0
    else:
        #光線が右側 左に法線
        position_angle = 1
    #全反射とそうでない場合を分ける #法線と光線の位置関係を追加してループできるようにする。
    if n1 < n2:
        #普通に透過
        #屈折角
        return calc_tan_after_transparency(n1, n2, sin_i, a, b, position_angle, 0)
    else:
        return calc_tan_after_transparency(n1, n2, sin_i, a, b, position_angle, 1)

#光線透過後の傾き計算
def calc_tan_after_transparency(n1, n2, sin_i, a, b, position_angle, reflection_type):
    if reflection_type == 0:
        sita_r = np.arctan((n1/n2)*sin_i)
        #水平軸と円の接線の法線が成す角
        c = 0
        d = 1
        if position_angle == 1:
            #屈折角+水平軸と円の接線の法線が成す角
            return abs(np.arctan(np.tan(abs(a*d - b*c)/abs(a*c - b*d))) + sita_r), 0
        else: #position_angle = 0
            return abs(np.arctan(np.tan(abs(a*d - b*c)/abs(a*c - b*d))) - sita_r), 0
    else:
        c = 0
        d = 1
        #臨界角の定義
        sita_h = abs(np.arcsin(n2/n1))
        sita_i = abs(np.arcsin(sin_i))
        if sita_i < sita_h : #ここいらないけどどりあえず処理追うときように残してある
            #全反射せずに透過する場合
            return calc_tan_after_transparency(n1, n2, sin_i, a, b, position_angle, 0)
        else:
            #全反射する場合記述
            sita_r = abs(np.arcsin(sin_i))
            if position_angle == 0: #注意間違っているかも
                return abs(np.pi - (sita_r - np.arctan(np.tan(abs(a*d - b*c)/abs(a*c - b*d))))), 1
            else:
                return abs(sita_r - np.arctan(np.tan(abs(a*d - b*c)/abs(a*c - b*d)))), 1
            
    
    
    
#楕円の式 x^2/a^2 + y^2/b^2 = 1 はこの変数で定義してね 
#a1,b1を外周の円 ,a2,b2を内周の円とする

#外で回し続けたほうが良いかも
while True: #need to change x,yが外周により外のとき
    x, y = calc_refraction(n1, n2, x1, y1, x2, y2, a2, b2, a1, b1)
    
    #最後にY軸となす角度を調べれば良さそう
    calc_angle(n1, n2, x1, y1, x2, y2, a1, b1)
    #外に出られる入射角だったら,その座標とY軸となす角度を記録
    if :
        break
        
#スリットらへんの処理

SyntaxError: invalid syntax (<ipython-input-1-98c7a5b459e2>, line 24)

12/20
現状回転無理そう、一旦一方向からの楕円だけで考える
- 進捗
- 楕円上側の内部に透過する場合の座標までとることができた

- 次のTodo
    - 光線の傾きと法線の位置関係を確かめれば割と簡単に角度の状態を記述できそうかもしれん　◯
    - 座標をキャッチするとき光線の傾きの正負で考えていったほうがx<0とかで考えるより楽かも ◯

12/22
- 進捗
    - 内部→外部の透過,全反射のときの光線の傾きは求められるようになった
  
- 次のTodo
    - y2<0の場合を記述
    
12/23
y2の部分で場合分けしてるけど,y2-y1とかにしたほうが向きが定義できて良いかもしれん→y<0だと-がついてくるのでy1y2を入力としてベクトルを定義した法が良いかも
先に受光学系実装する

## 受光部分を作る

- 緑色の部分</br>

<img src="./simulation.png" width="500"/>


- 受光できるかできないかを表す関数の実装
    - 入力：直線(傾き,切片),一応外周の座標
    - 出力：強度,角度,座標
    
スリット透過後は全てレンズによって一点に集まる
→スリット一枚目二枚目の間はほぼないとして,一枚目を基準として細管との距離が必要そう

- どうやって_sitaから光線の向きを求めるか light_receiving
    - そもそも受光はできるからそこに合わせてスリットを決めてったほうがいい？
        - あらかじめ座標はslit1 , slit2　通過時の座標は出しておく

In [ ]:
#まずslitの位置を定義 下準備
#円管とスリットの距離の定義が必要
distance1 = 1
distance2 = 1

x1y1 = np.zeros((900,2))
x2y2 = np.zeros((900,2))
s_sita = np.zeros(1801)
for sita in [tmp*0.1 for tmp in range(180, 270)]:
    x1 = distance1*np.cos(np.deg2rad(sita))
    y1 = distance1*np.sin(np.deg2rad(sita))
    x2 = distance2*np.cos(np.deg2rad(sita))
    y2 = distance2*np.sin(np.deg2rad(sita))
    similar_sita = abs(np.arctan(np.tan(abs(x1)/abs(y1))))
    x1y1[i][0] = x1
    x1y1[i][1] = y1
    x2y2[i][0] = x2
    x2y2[i][1] = y2
    s_sita[i] = similar_sita

In [1]:
#透過角 を求めるための回転装置をもしたシミュレーション
#90度だけで良さそう
#全ての直線データをぶちこむ
#外周座標は最後の透過角を求めるために必要。


def light_receiving(gradient ,intercept, x_k, y_k):        
    #スリット幅を入力
    s1 = 1
    s2 = 1
    #ある角度における光線の本数カウント用
    count_light = [0] * 900
    count_x = [0] * 900
    count_y = [0] * 900
    t_angle = [0] * 900
    
    for g, i, xk, yk in zip(gradient, intercept, x_k, y_k):
        for j in range(900): #90度分の回転を表す
            #スリットの傾きと,光線の交点
            a = -np.tan(s_sita[j])
            b = x1y1[j][1] - a*x1y1[j][0]
            x = (i - b)/(a - g)
            #スリット幅(斜めver)
            s_d = s1*np.cos(abs(s_sita[j]))/2
            #1つ目のスリットを超えられるかどうか
            if x < x1y1[j][0] + s_d and x1y1[j][0] - s_d < x:
                #2つめの条件に変更
                b = x2y2[j][1] - a*x2y2[j][0]
                x = (i - b)/(a - g)
                s_d = s2*np.cos(abs(s_sita[j]))/2
                #2つ目のスリットを超えられるかどうか
                if x < x2y2[j][0] + s_d and x2y2[j][0] - s_d < x:
                    #光の本数をカウント
                    count_light[j] += 1
                    #透過角算出用のx,yもおいておきたい→平均orとりあえずmin,maxの2つをキープしておいて考える。
                    #怪しい、場合によってはmaxminが大幅に離れてる？→ここはどのへんに値が偏っているのかを見てもいいかも一旦保留
                    count_x[j] += xk
                    count_y[j] += yk
    
    
    #jごとの透過角を求める必要がある(横軸)
    for j in range(900):
        #ある角度jのときのスリットを透過する外周の点の平均
        x_g = count_x[j]/count_light[j]
        y_g = count_y[j]/count_light[j]
        x_slit2 = x2y2[j][0]
        y_slit2 = x2y2[j][1]
        a = y_slit2 - y_g
        b = x_g - x_slit2
        c = 1
        d = 0
        t_angle[j] = np.arctan(np.tan(abs(a*d - b*c)/abs(a*c + b*d)))
        
        
        
                    

    
    

In [ ]:
#透過距離用のx軸移動